# Reaction Time Prediction Model - Model Comparison

## Objective
Predict human reaction time (PVT test) based on physiological and lifestyle factors.

**Dataset**: 64 samples, 7 original features (Gender, Sleep, Heart rate, Caffeine, O2 saturation, Stress, Age)

---

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

## 2. Load and Prepare Data

In [2]:
# Load dataset
df = pd.read_csv('Expérience 2 - Feuil1.csv')

# Remove unnecessary columns
df = df.drop(['GLUCOSE (mg/dL)', 'PARTICIPANT'], axis=1)

# Encode gender (H=1, F=0)
le = LabelEncoder()
df['SEXE'] = le.fit_transform(df['SEXE'])

## 3. Feature Engineering

Create new features by combining existing ones to capture interactions and non-linear relationships.

### Why These Features?

**1. SLEEP_STRESS = Sleep × Stress**
- **Combined effect**: Low sleep + high stress is worse
- **Expected**: More sleep with high stress = better RT; Less sleep with high stress = worse RT

**2. HEART_STRESS = Heart Rate × Stress**
- **Expected**: High heart rate + high stress Could improve RT (alertness) OR worsen it (anxiety)

**3. AGE_SLEEP = Age × Sleep**
- **Age-dependent sleep effect**: Older people need quality sleep more than younger people
- **Expected**: Young + low sleep = less impact; Old + low sleep = bigger impact

**4. CAFFEINE_SLEEP = Caffeine ÷ (Sleep + 1)**
    (+1)To prevent division by zero if sleep hours = 0.

- **Compensatory behavior**: People use caffeine to compensate for poor sleep
- **Relative dependency**: 200mg caffeine after 4h sleep ≠ 200mg after 8h sleep
- **Expected**: High ratio = trying to compensate for tiredness

**5. SLEEP_SQ = Sleep²**
- **Diminishing returns**: 8h→9h sleep improvement ≠ 4h→5h improvement
- **Non-linear relationship**: Sleep doesn't affect RT linearly

**6. AGE_SQ = Age²**
- **Non-linear aging**: RT decline accelerates with age (not constant)
- **Expected**: 20→30 age change ≠ 40→50 age change in RT impact

**7. STRESS_SQ = Stress²**
- **Exponential impact**: Moderate stress (30%) ≠ 2× impact of high stress (60%)

**8. SLEEP_DEFICIT = 1 if Sleep < 7, else 0**
- **Clinical threshold**: 7 hours is recommended minimum
- **Binary switch**: There's a "cliff" effect below 7h
- **Expected**: People with <7h sleep are in a different performance category

**9. HIGH_STRESS = 1 if Stress > 40%, else 0**
- **Threshold effect**: Moderate stress (20-40%) ≠ high stress (>40%)
- **Clinical relevance**: >40% biological stress is considered elevated
- **Categorization**: Separates "stressed" from "not stressed" individuals

In [3]:
# Create interaction and polynomial features
df['SLEEP_STRESS'] = df['SOMMEIL (H)'] * df['STRESS BIOLOGIQUE %']
df['HEART_STRESS'] = df['HEART RATE (BPM)'] * df['STRESS BIOLOGIQUE %']
df['AGE_SLEEP'] = df['AGE'] * df['SOMMEIL (H)']
df['CAFFEINE_SLEEP'] = df['CAFFEINE (mg)'] / (df['SOMMEIL (H)'] + 1)
df['SLEEP_SQ'] = df['SOMMEIL (H)'] ** 2
df['AGE_SQ'] = df['AGE'] ** 2
df['STRESS_SQ'] = df['STRESS BIOLOGIQUE %'] ** 2
df['SLEEP_DEFICIT'] = np.where(df['SOMMEIL (H)'] < 7, 1, 0)
df['HIGH_STRESS'] = np.where(df['STRESS BIOLOGIQUE %'] > 40, 1, 0)

## 4. Prepare Features and Target

In [4]:
# Separate X and y
X = df.drop('PVT : TEMPS DE REACTION (ms)', axis=1)
y = df['PVT : TEMPS DE REACTION (ms)']

## 5. Evaluation Metrics Explanation

### 1. MAE (Mean Absolute Error)
**Formule:**
$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

**Interprétation:**
- Mesure l'erreur moyenne absolue entre les valeurs prédites et réelles
- **Plus le MAE est faible, meilleur est le modèle**
- Unité: même unité que la variable cible (ms dans notre cas)
- **Bon résultat:** MAE < 10% de la moyenne de la variable cible

### 2. R² (Coefficient de détermination)
**Formule:**
$$R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$$

**Interprétation:**
- Mesure la proportion de variance expliquée par le modèle
- **Plage:** -∞ à 1 (négatif = très mauvais, 0 = modèle naïf, 1 = parfait)
- **Plus le R² est proche de 1, meilleur est le modèle**
- **Bon résultat:**
  - R² > 0.7 = Bon modèle
  - R² > 0.8 = Très bon modèle
  - R² > 0.9 = Excellent modèle

### 3. MRE (Mean Relative Error)
**Formule:**
$$MRE = \frac{1}{n} \sum_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{y_i} \times 100$$

**Interprétation:**
- Mesure l'erreur relative moyenne en pourcentage
- **Plus le MRE est faible, meilleur est le modèle**
- **Bon résultat:**
  - MRE < 5% = Excellent
  - MRE < 10% = Très bon
  - MRE < 15% = Acceptable
  - MRE > 20% = Nécessite amélioration

## 6. Train Models with Leave-One-Out Cross-Validation

### Validation: Leave-One-Out CV
- Uses maximum data (trains on 63, tests on 1, repeats 64 times)
- Most reliable for small datasets

We will compare 4 models:
1. **ElasticNet** - Linear model with regularization
2. **Gradient Boosting** - Ensemble of decision trees
3. **Random Forest** - Parallel ensemble of trees
4. **XGBoost** - Optimized gradient boosting

In [5]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dictionary to store results
results = {}
loo = LeaveOneOut()

### Model 1: ElasticNet

In [6]:
print("=" * 60)
print("Training ElasticNet...")
print("=" * 60)

# Create and train model
elastic_model = ElasticNet(alpha=1, l1_ratio=0.5, max_iter=10000, random_state=42)
y_pred_elastic = cross_val_predict(elastic_model, X_scaled, y, cv=loo)

# Calculate metrics
mae_elastic = mean_absolute_error(y, y_pred_elastic)
r2_elastic = r2_score(y, y_pred_elastic)
mre_elastic = np.mean(np.abs((y - y_pred_elastic) / y)) * 100

# Train final model on all data
elastic_model.fit(X_scaled, y)

results['ElasticNet'] = {'MAE': mae_elastic, 'R²': r2_elastic, 'MRE (%)': mre_elastic}
print(f"MAE: {mae_elastic:.4f}")
print(f"R²:  {r2_elastic:.4f}")
print(f"MRE: {mre_elastic:.4f}%")

Training ElasticNet...
MAE: 33.3544
R²:  0.0428
MRE: 8.6011%


### Model 2: Gradient Boosting

In [7]:
print("\n" + "=" * 60)
print("Training Gradient Boosting Regressor...")
print("=" * 60)

gb_model = GradientBoostingRegressor(
    n_estimators=120,
    max_depth=4,
    learning_rate=0.09,
    subsample=0.8,
    min_samples_split=10,
    min_samples_leaf=4,
    loss='huber',
    alpha=0.9,
    random_state=123
)

y_pred_gb = cross_val_predict(gb_model, X_scaled, y, cv=loo)

mae_gb = mean_absolute_error(y, y_pred_gb)
r2_gb = r2_score(y, y_pred_gb)
mre_gb = np.mean(np.abs((y - y_pred_gb) / y)) * 100

gb_model.fit(X_scaled, y)

results['Gradient Boosting'] = {'MAE': mae_gb, 'R²': r2_gb, 'MRE (%)': mre_gb}
print(f"MAE: {mae_gb:.4f}")
print(f"R²:  {r2_gb:.4f}")
print(f"MRE: {mre_gb:.4f}%")


Training Gradient Boosting Regressor...
MAE: 36.0774
R²:  0.0325
MRE: 9.3593%


### Model 3: Random Forest

In [8]:
print("\n" + "=" * 60)
print("Training Random Forest Regressor...")
print("=" * 60)

rf_model = RandomForestRegressor(
    n_estimators=120,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=123,
    n_jobs=-1
)

y_pred_rf = cross_val_predict(rf_model, X_scaled, y, cv=loo)

mae_rf = mean_absolute_error(y, y_pred_rf)
r2_rf = r2_score(y, y_pred_rf)
mre_rf = np.mean(np.abs((y - y_pred_rf) / y)) * 100

rf_model.fit(X_scaled, y)

results['Random Forest'] = {'MAE': mae_rf, 'R²': r2_rf, 'MRE (%)': mre_rf}
print(f"MAE: {mae_rf:.4f}")
print(f"R²:  {r2_rf:.4f}")
print(f"MRE: {mre_rf:.4f}%")


Training Random Forest Regressor...
MAE: 35.9991
R²:  -0.0289
MRE: 9.3207%


### Model 4: XGBoost

In [9]:
print("\n" + "=" * 60)
print("Training XGBoost Regressor...")
print("=" * 60)

xgb_model = XGBRegressor(
    n_estimators=120,
    max_depth=4,
    learning_rate=0.09,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=123,
    n_jobs=-1
)

y_pred_xgb = cross_val_predict(xgb_model, X_scaled, y, cv=loo)

mae_xgb = mean_absolute_error(y, y_pred_xgb)
r2_xgb = r2_score(y, y_pred_xgb)
mre_xgb = np.mean(np.abs((y - y_pred_xgb) / y)) * 100

xgb_model.fit(X_scaled, y)

results['XGBoost'] = {'MAE': mae_xgb, 'R²': r2_xgb, 'MRE (%)': mre_xgb}
print(f"MAE: {mae_xgb:.4f}")
print(f"R²:  {r2_xgb:.4f}")
print(f"MRE: {mre_xgb:.4f}%")


Training XGBoost Regressor...
MAE: 38.6688
R²:  -0.2197
MRE: 10.1461%
